# Prerequisites

Unless specified otherwise, we assume here that you have the virtual environment for the CBS fork of SynthSeg
installed properly.
To have direct access to the scripts used, please ensure you have installed this repository in the virtual environment by
running the following in a terminal with activated environment from the project's folder:

```shell
pip install -e .
```

# Rundown of analysis, training, and prediction of multichannel segmentation

The whole process consists of several steps that need to be repeated when training networks for different contrasts or resolutions.
Here, we try to train a dedicated network for the AD/PCA highres data.
The input for the segmentation will be

- 500 um resolution scans of the whole head
- we train on multichannel data that consists of PD, R1, and R2s maps
- the plain input maps will be clipped and rescaled to suitable value ranges (because noisy regions contain large outliers that would reduce the range of meaningful contrast for the network to learn from)

To run the whole pipeline, the following steps were necessary:

1. Decide how many different contrasts you want as input for your network. Remember that for this approach the contrasts need to be perfectly co-registered. At the moment, I believe this makes the most sense when you're using multiple echoes or, like here, quantitative maps that show different features.
2. Inspect the images and find suitable value ranges for all contrasts. These ranges are then used to clip and rescale the maps for further processing.
3. For at least one set of input images, you need a segmentation. It doesn't need to be a perfect segmentation, but it will be used to calculate the regional gray value statistics for each segmentation class.
4. From this analysis, a dedicated _generator configuration_ is created for you. It contains the specification for the random distribution for every segmentation class. This specification is used to create random synthetic maps that should resemble your original data as close as possible. In the case here, this will not be perfect because maps like R2s have gray value distributions in some regions that can not really be modeled with a Gaussian.
5. The generator configuration is used to create the training data for the neural network. There are two options: You can create the training data on-the-fly, or you can create them beforehand and store them in a TensorFlow format that is easy to deserialize. The latter approach has another advantage. You can train different network parametrizations with the same training data and compare the performance.
6. With the training data (or the generation configuration), you derive a training configuration that further specifies the used network model and its hyperparameters.
7. During the training, we write out the weights and biases of the model as snapshots. These can be loaded back if you want to create a network for prediction (aka running the actual segmentation)


## 1. & 2. Used Contrasts and Value Ranges

The first of training a new network is to decide which contrasts to use and to clip and scale them to an appropriate range.
SynthSeg (or networks in general) are happy with values [0, 1] or reasonable other ranges.
However, if your images contain large outlier noise pixel, e.g. your main image information is in the range of [0, 1000] and there are
outlier noise pixel with values > 20000, scaling might compress the important information too much.
Also, the ranges in your image depend largely on the outlier noise and might hinder a good training and prediction.

Therefore, the first step is to look into your maps/images and make a decision for good value ranges.
In this experiment, we want to use three quantitative maps that are combined into a multichannel image and train the segmentation on it.
We decided on the following maps, and we decided to use the following ranges:

1. Proton Density (PD): Original gray value range: [-173181.7, 1933.817], clip to [0, 200], rescale to [0, 1]
2. Relaxation Rate R1: Original gray value range: [0, 3.03], clip to [0, 3], rescale to [0, 1]
3. R2s: Original gray value range: [-720.689, 1085.598], clip to [0, 200], rescale to [0, 1]

The reason for clipping the maps is that both a "density" and the "relaxation rates" are non-negative by definition and have a much smaller
range.
The driver of appearing small and large outliers are the calculations of these values which do not work correctly on, e.g., noisy background.

## 2. & 3. Running the Region Analysis and Create the Generator Config

For analyzing the different regions of an example image, we need a segmentation.
The goal is to have a rough (good enough) segmentation so that we can inspect each label and calculate statistics on the gray level distribution within
each segmented region.
Therefore, we take an image from our example and run it through standard SynthSeg and use this segmentation to analyze each labeled region.
The only requirements are (a) you should get a half-decent segmentation and (b) the image you're using for the initial segmentation needs to be co-aligned
(registered, in the same space) as the images you want to analyze.

First, let's try a simple T1 map that we clip and rescale to have good values:

In [ ]:
from SynthSeg.analysis.contrast_analysis import clip_and_rescale_nifti

input_dir = "/Users/pscheibe/Documents/tmpData/SynthSeg/01"
output_dir = "/Users/pscheibe/Documents/tmpData/SynthSeg/01/analysis"

initial_t1 = f"{input_dir}/T1w_WOLSfit.nii"
t1_for_segmentation = f"{output_dir}/T1w_WOLSfit_rescaled_for_segmentation.nii"
clip_and_rescale_nifti(
    nifti_file=initial_t1,
    out_file=t1_for_segmentation,
    min_clip=0.0,
    max_clip=2000,
    min_out=0.0,
    max_out=1.0
)

Now, you use the default SynthSeg to get a 1mm segmentation. For that:

1. Clone the original repo again (or switch and pull your existing SynthSeg repo)
2. Download the necessary model snapshots that are linked in the [installation guide](https://github.com/BBillot/SynthSeg?tab=readme-ov-file#installation)
3. Create the necessary Python environment and activate it

Then you can segment the scaled T1w image from above.
Check the command line arguments for SynthSeg.
On my Mac here without GPU, I use:


```shell
python ./scripts/commands/SynthSeg_predict.py \
       --i $output_dir/T1w_WOLSfit.nii \
       --o $output_dir/T1w_WOLSfit_synthseg.nii \
       --robust --cpu --threads 8
```

### Running the Region Analysis

The region analysis needs the following things:

- A segmentation label image for the brain images you want to analyze
- One or more contrast brain images that are going to be analyzed
- Clipping and scaling parameters for each contrast image
- (Optional) statistical settings that influence how each region is analyzed

The region analysis creates the following output:

- A template "brain generator config" named `_generator.yml` that defines the statistical values for the gray levels of each region
- A clipped and scaled Nifty image for each input contrast image
- A combined volume that has all clipped and scaled contrasts in different channels. This is basically similar to what the brain generator will create as training input.

Using the created decent-enough-segmentation, the input contrasts and the values for clipping, you can call the analysis script as follows:

```shell
synthSeg-mpm-analyse \
  --label_file "$output_dir/T1w_WOLSfit_synthseg.nii" \
  --contrast_files "$input_dir/PD.nii" "$input_dir/R1.nii" "$input_dir/R2s.nii" \
  --output_dir "$output_dir" \
  --clip_min 0 0 0 --clip_max 200 2 200
```

Please also look at `synthSeg_mpm_analyse --help` to see all options.
The contrast images and the combined image are scaled into the range [0,1] per default.
When you prepare a training run, it is very helpful to provide a `template_generator_config` for the analysis call.
You can fix all necessary paths and other settings because the analysis script will only touch options that handle the labels, channels and their gray value distributions.


## 4. & 5. Creating the Training Data

Let's start off by creating a `_generator.yml` directly in our experiment folder and set the gray value distribution parameters
extremely narrow.
Additionally, we use a config template where all augmentations are turned off.
With this, we can generate test data and compare it with the maps that we analyzed.
The gray levels should roughly follow what you find in the MPMs.

```
synthseg_data_dir="/Users/pscheibe/PycharmProjects/SynthSeg/data/cbs/test_256_pd_r1_r2s" 

synthSeg-mpm-analyse \
  --label_file "$output_dir/T1w_WOLSfit_synthseg.nii" \
  --contrast_files "$input_dir/PD.nii" \
    "$input_dir/R1.nii" \
    "$input_dir/R2s.nii" \
  --output_dir "$synthseg_data_dir" \
  --clip_min 0 0 0 --clip_max 200 2 200 \
  --template_generator_config "$synthseg_data_dir/analysis_template_no_augment.yml" \
  --range_brackets 0.9999 1.0001 0.001 0.002
```

Next, we can create test data as follows:

```
test_data_dir="/Users/pscheibe/PycharmProjects/SynthSeg/output"

synthSeg-generate-synth-images \
  --config_file "$synthseg_data_dir/_generator.yml" \
  --output_dir "$test_data_dir" \
  --count 5
```

If this looks satisfactory, all augmentations like random rotations, non-linear transformations, bias-field, etc. need to be turned back on.
Also, the `range_brackets` option should have reasonable values to give the generator enough freedom to create random contrasts while still following
the distributions found in our maps.
One important step: You might want to adjust the mean and std range for the first entry (background) in each channel.
Roughly speaking, the MPM backgrounds contain many artifacts and by having a wide random range of gray values, the network hopefully learns to not use these regions for anything.



## 6. Training Configuration and Training on HPC

# Importing the necessary libraries and modules


In [1]:

import os
import tensorflow as tf

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

import commands.cbs_predict as pred
from SynthSeg.training_options import TrainingOptions
from SynthSeg.brain_generator_options import GeneratorOptions


training_config_file = "/home/patrick/Documents/CBS/DataShare/SynthSeg/Models/202401/training.yml"
generator_config_file = "/home/patrick/Documents/CBS/DataShare/SynthSeg/Models/202401/generator.yml"

# Continue with the rest of your code...
# Continue with the prediction part of your code.

training_options = TrainingOptions.load_yaml(training_config_file)
generator_options = GeneratorOptions.load_yaml(generator_config_file)

training_options = training_options.convert_lists_to_numpy()
generator_options = generator_options.convert_lists_to_numpy()

predict_options = pred.PredictOptions(
    training_config=training_config_file,
    path_images="/home/patrick/Documents/CBS/DataShare/SynthSeg/Models/202401/tfrecord2nifty/images",
    gt_folder="/home/patrick/Documents/CBS/DataShare/SynthSeg/Models/202401/tfrecord2nifty/labels",
    path_model="/home/patrick/Documents/CBS/DataShare/SynthSeg/Models/202401/dice_063.h5",
    path_segmentations="/home/patrick/Documents/CBS/DataShare/SynthSeg/Models/202401/predictions/segmentations/",
    path_posteriors="/home/patrick/Documents/CBS/DataShare/SynthSeg/Models/202401/predictions/posteriors/",
    target_res=training_options.target_res,
)

2024-01-11 13:27:14.206589: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-11 13:27:14.279029: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-11 13:27:15.595196: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, re

No GPU found


In [6]:

pred.predict(
    predict_options.path_images,
    predict_options.path_segmentations,
    predict_options.path_model,
    generator_options.output_labels,
    n_neutral_labels=generator_options.n_neutral_labels,
    names_segmentation=predict_options.names_segmentation,
    path_posteriors=predict_options.path_posteriors,
    path_resampled=predict_options.path_resampled,
    path_volumes=predict_options.path_volumes,
    min_pad=predict_options.min_pad,
    cropping=predict_options.cropping,
    target_res=predict_options.target_res,
    gradients=predict_options.gradients,
    flip=predict_options.flip,
    topology_classes=predict_options.topology_classes,
    sigma_smoothing=predict_options.sigma_smoothing,
    keep_biggest_component=predict_options.keep_biggest_component,
    n_levels=training_options.n_levels,
    nb_conv_per_level=training_options.nb_conv_per_level,
    conv_size=training_options.conv_size,
    unet_feat_count=training_options.unet_feat_count,
    feat_multiplier=training_options.feat_multiplier,
    activation=training_options.activation,
    gt_folder=predict_options.gt_folder,
    evaluation_labels=predict_options.evaluation_labels,
    list_incorrect_labels=predict_options.list_incorrect_labels,
    list_correct_labels=predict_options.list_correct_labels,
    compute_distances=predict_options.compute_distances,
    recompute=predict_options.recompute,
    verbose=predict_options.verbose,
    use_original_unet=training_options.use_original_unet
)


predicting 1/8
1/1 [==============================] - 17s 17s/step
predicting 2/8    remaining time: 0:11:09
1/1 [==============================] - 15s 15s/step
predicting 3/8    remaining time: 0:09:16
1/1 [==============================] - 14s 14s/step
predicting 4/8    remaining time: 0:07:45
1/1 [==============================] - 14s 14s/step
predicting 5/8    remaining time: 0:06:11
1/1 [==============================] - 15s 15s/step
predicting 6/8    remaining time: 0:04:41
1/1 [==============================] - 15s 15s/step
predicting 7/8    remaining time: 0:03:07
1/1 [==============================] - 15s 15s/step
predicting 8/8    remaining time: 0:01:33
1/1 [==============================] - 14s 14s/step
evaluating 1/8


In [3]:
generator_options.generation_labels

array([  0,  14,  15,  16,  24,  72,  85, 502, 506, 507, 508, 509, 511,
       512, 514, 515, 516, 530,   2,   3,   4,   5,   7,   8,  10,  11,
        12,  13,  17,  18,  25,  26,  28,  30, 136, 137,  41,  42,  43,
        44,  46,  47,  49,  50,  51,  52,  53,  54,  57,  58,  60,  62,
       163, 164])

In [ ]:
import os
import tensorflow as tf
import numpy as np
import nibabel as nb
import SynthSeg.segmentation_model as M
import commands.tfrecord_to_nifti as tfnii
import itertools
from typing import List

# Uncomment this if you run out of GPU memory when predicting
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
print("Using GPU" if tf.test.gpu_device_name() else "Using CPU")

tfrecord_file = "/home/patrick/Documents/CBS/DataShare/SynthSeg/Models/202401/017132.tfrecord"
nifty_dir = "/home/patrick/Documents/CBS/DataShare/SynthSeg/Models/202401/tfrecord2nifty/"
net_file = "/home/patrick/Documents/CBS/DataShare/SynthSeg/Models/202401/dice_063.h5"
output_dir = "/home/patrick/Documents/CBS/DataShare/SynthSeg/Models/202401/predictions/cbs_data/"
# output_dir = "/home/patrick/Documents/CBS/DataShare/SynthSeg/Models/202401/predictions/training_data/"


def adjust_size(input_data: np.ndarray, desired_shape: List[int]):
    """
    Adjusts the size of the input data array to match the desired shape.

    It does it by either cropping or padding symmetrically.

    Args:
        input_data: The input data array.
        desired_shape: The desired shape for the input data array.

    Returns:
        The input data array adjusted to the desired shape.
    """
    current_shape = input_data.shape
    adjustments = [(ds - cs) / 2. for ds, cs in zip(desired_shape, current_shape)]
    pad_values = [(int(np.floor(adj)), int(np.ceil(adj))) if adj > 0 else (0, 0) for adj in adjustments]
    crop_values = [(int(np.floor(-adj)), -int(np.ceil(-adj))) if adj < 0 else (0, 0) for adj in adjustments]
    content_values = [(int(np.floor(adj)), int(cs + np.ceil(adj))) for adj, cs in zip(adjustments, current_shape)]
    # Pad / Crop the array for non-negative / negative adjustments (respectively)
    input_data_adjusted = np.pad(input_data, pad_width=pad_values, mode='constant', constant_values=0)
    input_data_adjusted = input_data_adjusted[
                          crop_values[0][0]:content_values[0][1],
                          crop_values[1][0]:content_values[1][1],
                          crop_values[2][0]:content_values[2][1]]

    return input_data_adjusted


def softmax(x: np.ndarray) -> np.ndarray:
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=-1, keepdims=True)


# Create nifti from a tfrecord file.
# tfnii.tfrecord_to_nifty(tfnii.Options(input_file=tfrecord_file, output_directory=nifty_dir))

print("Loading input data")
# nifty_file = f"{nifty_dir}images/017132_3.nii.gz"
nifty_file = "/home/patrick/Workspace/cbs/SynthSeg/data/cbs/test_256_pd_r1_r2s/combined.nii"
img = nb.load(nifty_file)
input_data = img.get_fdata()

# Prepare the network
print("Creating network")
net = M.unet(input_shape=(None, None, None, 3), n_labels=33)
net.load_weights(net_file, by_name=True)

current_shape = input_data.shape
print(f"Original shape {current_shape}")

# You want the size to be [512, 512, 512, 3]
desired_shape = [256, 256, 256, 3]

adjusted_data = adjust_size(input_data, desired_shape)
print("Storing original but cropped image")

adjusted_data = np.flip(np.transpose(adjusted_data, [2, 0, 1, 3]), 1)/np.max(adjusted_data)
epsilon = 1e-7  # define very close to 0 or 1
outlier_mask = np.abs(adjusted_data) < epsilon
adjusted_data[outlier_mask] = np.random.rand(np.count_nonzero(outlier_mask))/10.0  # replace with random values
outlier_mask = np.abs(adjusted_data-1.0) < epsilon
adjusted_data[outlier_mask] = np.random.rand(np.count_nonzero(outlier_mask))/10.0  # replace with random values

# nb.save(nb.Nifti1Image(adjusted_data, np.eye(4), img.header), f"{output_dir}image.nii")
# prediction = net.predict(np.expand_dims(adjusted_data, axis=0))

print("Saving Segmentation")
# nb.save(nb.Nifti1Image(np.squeeze(softmax(prediction)), np.eye(4), img.header), f"{output_dir}posteriors.nii")
# exit(0)

# for number, order in enumerate(itertools.permutations([0, 1, 2])):
#     print(f"Predicting permutation {number}: {order}")
#     new_order = list(order) + [3]
#     new_data = np.transpose(adjusted_data, new_order)
#     nb.save(nb.Nifti1Image(new_data, np.eye(4), img.header), f"{output_dir}image_{number}.nii")
#     # prediction = net.predict(np.expand_dims(new_data, axis=0))
#     # nb.save(nb.Nifti1Image(np.squeeze(prediction), img.affine, img.header),
#     #         f"{output_dir}posteriors_{number}.nii")


# Code with 90 degree rotations and transpositions
for number, order in enumerate(itertools.permutations([0, 1, 2])):
    new_order = list(order) + [3]
    rotated_data = adjusted_data
    for axes in itertools.combinations([0, 1, 2], 2):  # All combinations of 2 axes in 3D
        for k in range(2):  # 4 rotations for each combination of axes (0, 90, 180, 270 degrees)
            print(f"Predicting permutation {number} {axes} {k+1}")
            rotated_data = np.rot90(rotated_data, k+1, axes)
            new_data = np.transpose(rotated_data, new_order)
            nb.save(nb.Nifti1Image(new_data, np.eye(4), img.header), f"{output_dir}image_{number}_{axes}_{k+1}.nii")
            prediction = net.predict(np.expand_dims(new_data, axis=0))
            nb.save(nb.Nifti1Image(np.squeeze(softmax(prediction)), img.affine, img.header),
                    f"{output_dir}posteriors_{number}_{axes}_{k}.nii")
